In [1]:
import os
from datetime import datetime, timedelta, timezone
import shutil

In [2]:
from configs.config_object import ConfigObject
from grid_and_bathy import build_and_save_mitgcm_grid, get_dz_grid, get_grid, MitgcmGrid
from surface_forcings import extract_and_save_surface_forcings, download_weather_reanalysis, download_weather_forecast
from initial_conditions import download_profile_idronaut_datalakes, create_initial_temperature_from_measure_profile
from run_preprocessing import write_data_config_files, copy_template, remove_all_files_and_folders, write_size_config_files
from utils import modify_arguments, convert_binary_files

## Load config

In [3]:
config = ConfigObject('configs/config.json')
output_folder = './99-output/'
template_folder = './00-template_mitgcm/geneva/'

In [4]:
parsed_start_date = datetime.strptime(config.start_date, '%Y%m%d')
parsed_end_date = datetime.strptime(config.end_date, '%Y%m%d')
sim_duration_in_second = (parsed_end_date - parsed_start_date).total_seconds()

## Initialize output folder

In [5]:
remove_all_files_and_folders(output_folder)
copy_template(template_folder, output_folder)
config.write_metadata_to_file(os.path.join(output_folder, 'metadata_simulation.txt'))

## Get MITgcm grid

In [6]:
mitgcm_grid = get_grid(config.paths.grid_folder_path)

#In case the grid has not been created yet
build_and_save_mitgcm_grid(config.paths.grid_folder_path, 
                           config.Nx, 
                           config.Ny, 
                           config.grid_resolution, 
                           config.x0_ch1903, 
                           config.y0_ch1903, 
                           config.x1_ch1903, 
                           config.y1_ch1903)

## Build binary files

In [7]:
binary_data_folder = os.path.join(output_folder, 'binary_data')
if not os.path.exists(binary_data_folder):
    os.makedirs(binary_data_folder)
    print(f"Directory '{binary_data_folder}' created.")

Directory './99-output/binary_data' created.


**Get bathymetry**

In [8]:
#To create bathymetry, see notebook 'create_bathy'
shutil.copy(config.paths.bathy_path, os.path.join(binary_data_folder, 'bathy.bin'))

'./99-output/binary_data\\bathy.bin'

**Get surface forcings**

In [9]:
buffer = config.weather_download_buffer
if config.weather_model_type == 'reanalysis':
    download_weather_reanalysis(config.weather_api_base_url, 
                                config.start_date, config.end_date, 
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_results_from_api_folder)
    print('Finished downloading weather reanalysis')
elif config.weather_model_type == 'forecast':
    download_weather_forecast(config.weather_api_base_url, 
                                config.start_date,  
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_results_from_api_folder)
    print('Finished downloading weather forecasts')

Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20231201_20231202_T_2M.json
Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20231201_20231202_U.json
Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20231201_20231202_V.json
Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20231201_20231202_GLOB.json
Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20231201_20231202_RELHUM_2M.json
Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\data\COSMO reanalysis\raw_from_alplakes_api\20231201_20231202_PMSL.json
Data already exists: C:\Users\leroquan\Documents\00-Work_space\06-mit_gcm_geneva\da

In [10]:
extract_and_save_surface_forcings(binary_data_folder, 
                                  config.start_date, 
                                  config.end_date, 
                                  config.paths.raw_results_from_api_folder, 
                                  mitgcm_grid,
                                  config.weather_model_type)

Interpolating u10 to grid...
Saving u10 to file ./99-output/binary_data\u10.bin
Interpolating v10 to grid...
Saving v10 to file ./99-output/binary_data\v10.bin
Interpolating air temperature (atemp) to grid...
Saving atemp.
Interpolating atmospheric pressure (apress) to grid...
Saving apressure.
Interpolating relative humidity (relhum) to grid...
Computing specific humidity (aqh) from air temperature and cloud cover...
Saving aqh.
Interpolating shortwave radiation (swdown) to grid...
Saving swdown.
Interpolating cloud cover (CLCT) to grid...
Computing long wave radiation (lwdown) from air temperature and cloud cover...
Saving lwdown.


**Convert to little endian if necessary** (for cscs computation)

In [11]:
# TO DO : implement check to see whether the datatypes are correct
if config.endian_type == 'little_endian':
    convert_binary_files(os.path.join(output_folder, 'binary_data'), '>f8', '<f8')

### Get initial conditions

In [12]:
file_date, measured_temperature = download_profile_idronaut_datalakes(parsed_start_date)
print('Initial conditions from Idronaut file from ', file_date)
dz_grid = get_dz_grid(config.paths.dz_grid_csv_path)
shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, measured_temperature)

Initial conditions from Idronaut file from  2023-11-30 20:53:12.800000


### Get pickup files

In [13]:
# TO DO : copy pickup files from designated folder and iteration number

### Adapt config files

In [14]:
modify_arguments('!start_date!', [config.start_date], os.path.join(output_folder, 'run_config/data.cal'))

In [15]:
modify_arguments('!start_date!', [config.start_date], os.path.join(output_folder, 'run_config/data.exf'))

In [16]:
#TO DO: write output type (binary or netcdf -->   diag_mnc = .FALSE./.TRUE. in data.diagnostics)

In [17]:
write_data_config_files(os.path.join(output_folder, 'run_config/data'), shaped_temp_initial, sim_duration_in_second, dz_grid, config.grid_resolution)

In [18]:
write_size_config_files(os.path.join(output_folder, 'code/SIZE.h'), 
                        config.Px, 
                        config.Py, 
                        config.Nx, 
                        config.Ny, 
                        len(dz_grid.flatten()))

## Next steps

- Copy-paste the folders in "99-output" to the cscs server 
- Change #SBATCH --time=00:05:00 in run/submit-daint-short.sh
- Open CSCS putty console:

export MITGCM_ROOTDIR=/scratch/snx3000/aleroqua/MITgcm-checkpoint67z/

cd /scratch/snx3000/aleroqua/xx/build
$MITGCM_ROOTDIR/tools/genmake2 -mods ../code -mpi -enable=mnc -of ${MITGCM_ROOTDIR}/tools/build_options/linux_ia64_cray_archer
make depend
make -j 8

cd ..
rm -r run
mkdir run
cd run
ln -s ../pickup/* .
ln -s ../run_config/* .
cp ../build/mitgcmuv .

sbatch submit-daint-short.sh

### TO DO

In [19]:
# add selection of pickup file
# add selection of pickup frequency, diagnostic frequency
# add selection of output type (mnc = true or false)
# add automatic change of computing time? (submit-daint)